In [61]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = 100
import plotly
import plotly.express as px
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt

from ipywidgets import interact
import ipywidgets as widgets

from sklearn.utils import shuffle


In [2]:
import sys
sys.path.append('../src/')
import launcher
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (accuracy_score, f1_score, log_loss)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import svm

# 1) Some exploration

In [4]:
df = pd.read_excel("../data/01_raw/tarapam.xlsx")
df = df.rename(columns = {"Unnamed: 0" : "Sref"})
df_sref = pd.DataFrame(df["Sref"])
df = df.drop("Sref", axis=1)
df["Label"] = 0
df.loc[0:99,"Label"] = 1

**Compare the distribution of each feature between the two classes : phosphate (1) and sterile (0)**

In [7]:
@interact(
    col=widgets.Dropdown(
        options=df.columns,
        value=250,
        description='Val',
        disabled=False
    )
)

def f(col):    
    data=[
        go.Box(
            y=df[df["Label"]==e][col],
            name=f"Label {str(e)}",
            boxmean=True,
            ) for e in [0,1]
    ]
    
    fig = go.FigureWidget(data)
    fig.show()

interactive(children=(Dropdown(description='Val', options=(250, 255, 260, 265, 270, 275, 280, 285, 290, 295, 3…

**Trying some models without any feature selection**

In [13]:
# train test split
y = df["Label"]
X = df.drop("Label",axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=33)

**Fit and evaluate KNN model on data, without any feature selection**

In [18]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print(f"Train accuracy score : {accuracy_score(y_train, knn.predict(X_train))}")
print(f"Test accuracy score : {accuracy_score(y_test, y_pred)}")

Train accuracy score : 0.8938053097345132
Test accuracy score : 0.7142857142857143


**Fit and evaluate SVM model on data, without any feature selection**

In [19]:
scaler = StandardScaler()
x_train_svm = scaler.fit_transform(X_train)
x_test_svm = scaler.transform(X_test)
svm_clf = svm.SVC(kernel="rbf")
svm_clf.fit(x_train_svm, y_train)
y_pred_svm = svm_clf.predict(x_test_svm)
print(f"Train accuracy score : {accuracy_score(y_train, svm_clf.predict(x_train_svm))}")
print(f"Test accuracy score : {accuracy_score(y_test, y_pred_svm)}")

Train accuracy score : 0.8938053097345132
Test accuracy score : 0.7321428571428571


# 2) Visualize feature impact in classifying the samples

In [49]:
@interact(
    col1=widgets.Dropdown(
        options=df.columns,
        value=250,
        description='feature 1',
        disabled=False
    )
)

def f(col1): 
    @interact(
    col2=widgets.Dropdown(
        options=df.columns,
        value=500,
        description='feature 2',
        disabled=False
        )
    )
    def g(col2): 
        @interact(
        col3=widgets.Dropdown(
            options=df.columns,
            value=1000,
            description='feature 3',
            disabled=False
            )
        )
        def h(col3):
            df_plot = df[[col1,col2,col3,"Label"]]
            df_plot = df_plot.rename(columns = {col1:str(col1), col2:str(col2), col3:str(col3)})
            fig = px.scatter_3d(df_plot,
                    x=str(col1),
                    y=str(col2),
                    z=str(col3),
                    color='Label', 
                    )
            fig.show()

interactive(children=(Dropdown(description='feature 1', options=(250, 255, 260, 265, 270, 275, 280, 285, 290, …

# 3) Analysinf main run results

In [117]:
#model = launcher.run()